In [1]:
# GOAL: convert whole paragraph from biblegateway url to colored paragraph in English, 
#       depending on the length of parsing text

import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re

# load English bible book from biblegateway.com
url = input("Enter url from biblegateway in NASB1995:")
# Example: https://www.biblegateway.com/passage/?search=Luke+1&version=NASB1995
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')

# initial clean 
paragraph = []
for i in soup.find_all("p"):
    para = i.get_text()
    para = para.replace("\xa0","")
    paragraph.append(para)
paragraph = ''. join(paragraph)
paragraph = paragraph.split()

# remove "()","[]" in the text
# getting rid of alphabet in "(A)","[a]" 
for i in range(len(paragraph)):
    paragraph[i] = re.sub("\(.*?\)","",paragraph[i])
for i in range(len(paragraph)):
    paragraph[i] = re.sub("\[.*?\]","",paragraph[i])

# separate numeric from text
a = []
for i in range(len(paragraph)):
    parts = re.findall('\d+|\D+', paragraph[i])
    a.append(parts)
paragraph = [item for parts in a for item in parts]

# get rid of unwanted texts at the bottom
for i in range(len(paragraph)):
    if "®" in paragraph[i]:
        break
del paragraph[i:len(paragraph)]

for j in reversed(range(len(paragraph))):
    if "." in paragraph[j]:
        period_index = paragraph[j].index(".")
        paragraph[j] = paragraph[j][:period_index+1]
        break
del paragraph[j+1:len(paragraph)]
paragraph[0] = '1'

#list of numeric
num = ""
num_lst = []
for i in range(len(paragraph)):
    if (paragraph[i].isdigit()):
        num = paragraph[i]
        num_lst.append(num)

# list of index of each numeric, and the last index of the text
def get_index(paragraph,num_lst):  
    index_list = [] 
    for i in range(len(num_lst)):
        if num_lst[i] in paragraph:
            index = paragraph.index(str(num_lst[i]))
            index_list.append(index)
    last = len(paragraph)
    index_list.append(last)
    return index_list
index_list = get_index(paragraph,num_lst)

def get_index_only(paragraph,num_lst):  
    index_list_only = []
    for i in range(len(num_lst)):
        if num_lst[i] in paragraph:
            index = paragraph.index(str(num_lst[i]))
            index_list_only.append(index)
    return index_list_only

# list of each verse
def word_to_sentence(paragraph,index_list): 
    verse = []
    paragraph_English = []
    for i in range(len(index_list)-1):
        start = index_list[i]
        end = index_list[i+1]
        verse.append(paragraph[start:end])
        verse = [' '.join(j) for j in verse]
        paragraph_English.append(verse)
        verse = []
    return paragraph_English
paragraph_English = word_to_sentence(paragraph,index_list)

Enter url from biblegateway in NASB1995:https://www.biblegateway.com/passage/?search=Luke+2&version=NASB1995


In [2]:
from itertools import zip_longest

# load bible book from blueletterbible.org
parsing_text = input("Enter txt file name:")
# Example: luke1:7-8.txt
with open(parsing_text, 'r') as f:
    parsing = f.readlines()

# arrange text into group of 3: (English, Greek, Parsing Code)
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)
parsing = list(grouper(parsing,3,'-'))

# get rid of \n
for i in range(len(parsing)):
    parsing[i] = list(parsing[i])
    for j in range(len(parsing[i])):
        parsing[i][j] = parsing[i][j].replace("\n","")
    
# group text by parsing code
code = []
for a in parsing:
    if a[2] not in code:
        code.append(a[2])
        code.append([])
for b in code:
    for a in parsing:
        if b == a[2]:
            code[code.index(b)+1].append([a[0], a[1]])
# for no in range(0,len(code),2):
#     print([code[no]] + [a for a in code[no+1]])

# apply color   
RESET = '\033[0m'
def get_color_escape(r, g, b, background=True):
    return '\033[{};2;{};{};{}m'.format(48 if background else 38, r, g, b)

COLOR = {
    "NOM": get_color_escape(0, 255, 0),
    "ACC": get_color_escape(255, 0, 0),
    "DAT": get_color_escape(128, 128, 128),
    "GEN": get_color_escape(0, 128, 128),
    "PREP": get_color_escape(128, 128, 0),
    "VERB": get_color_escape(255, 255, 0),
    "PART": get_color_escape(0, 255, 255),
    "INFI": get_color_escape(255, 0, 255),
    "CONJ": "\033[4m",
    "BLACK": "\033[0m"
}

# define function to convert parsing code to color code
def mini_code_to_color(code):  
    if "D" in code:
        return COLOR["DAT"]
    elif "G" in code:
        return COLOR["GEN"]
    elif "V" in code:
        return COLOR["VERB"]
    if i+2 < len(code):
        if "N" in code[i+2]:
            return COLOR["NOM"]
        elif "A" in code[i+2]:
            return COLOR["ACC"]  
        else:
            return COLOR["BLACK"]
    else:
        return COLOR["BLACK"]

def code_to_color(code):  
    if "PREP" in code:
        return COLOR["PREP"]
    if "CONJ" in code:
        return COLOR["CONJ"]
    for i in range(len(code)):
        if "D" in code:
            return COLOR["DAT"]
        if "G" in code:
            return COLOR["GEN"]
        if "V" in code:
            if "PAN" in code:
                return COLOR["INFI"]
            elif "PMN" in code:
                return COLOR["INFI"]
            elif "PPN" in code:
                return COLOR["INFI"]
            elif "AAN" in code:
                return COLOR["INFI"]
            elif "AMN" in code:
                return COLOR["INFI"]
            elif "APN" in code:
                return COLOR["INFI"]
            elif "P-" in code[i+4:i+6]:
                return COLOR["PART"]
            else:
                return COLOR["VERB"]
        if "N" in code[i]:
            if i+2 < len(code):
                if "N" in code[i+2]:
                    return COLOR["NOM"]
                if "A" in code[i+2]:
                    return COLOR["ACC"]
                else:
                    mini_code_to_color(code[i+2])
            else:
                return COLOR["NOM"]
        elif "A" in code[i]:
            if i+2 < len(code):
                if "A" in code[i+2]:
                    return COLOR["ACC"]
                if "N" in code[i+2]:
                    return COLOR["NOM"]
                else:
                    mini_code_to_color(code[i+2])
            else:
                return COLOR["ACC"]
        elif "T" in code[i]:
            if i+2 < len(code):
                if "A" in code[i+2]:
                    return COLOR["ACC"]
                if "N" in code[i+2]:
                    return COLOR["NOM"]
                else:
                    mini_code_to_color(code[i+2])
            else:
                return COLOR["BLACK"]
        elif "P" in code[i]:
            if i+2 < len(code):
                if "A" in code[i+2]:
                    return COLOR["ACC"]
                if "N" in code[i+2]:
                    return COLOR["NOM"]
                else:
                    mini_code_to_color(code[i+2])
            else:
                return COLOR["BLACK"]
    else:
        return COLOR["BLACK"]
    
# colored text in English(forward)
def forward_English(parsing):
    colored_English = ""
    for i in range(len(parsing)):
        colored_English += code_to_color(parsing[i][-1])+parsing[i][0]+RESET + " "
    return colored_English

# colored text in Greek(reversed)
def reversed_Greek(parsing):
    colored_Greek = ""
    for i in range(len(parsing)):
        colored_Greek += code_to_color(parsing[i][-1])+parsing[i][1]+RESET + " " 
    return colored_Greek_reversed

# let user enter starting verse number and ending verse number
number_start = int(input("Enter start verse:"))
number_end = int(input("Enter end verse:"))
user_start = number_start
user_end = number_end

Enter txt file name:Luke2:1-21.txt
Enter start verse:1
Enter end verse:21


In [3]:
# get token from txt file
def token_English(parsing):
    parsing_English = []
    for i in range(len(parsing)):
        parsing_English.append(parsing[i][0])
    return parsing_English

# arrange TE_combine so that it fits regex
TE = token_English(parsing)
TE_select = []
for i in range(len(TE)):
    if " " in TE[i]:
        TE_select.append(TE[i])
TE_combine = ""
for i in range(len(TE_select)):
    TE_combine = TE_combine + "|" + TE_select[i]
    i += 1
TE_combine = TE_combine[1:]

index_list_only = get_index_only(paragraph,num_lst)
start = index_list_only[user_start-1]
end = index_list_only[user_end]
user_paragraph = paragraph[start:end]
user_index_list = get_index(user_paragraph,num_lst)
user_paragraph = word_to_sentence(user_paragraph,user_index_list)

# list of verses with correct tokens that match the mapping text file
itr_lst = []
all_lst = []
for i in range(len(user_paragraph)):
    find_regex = r''+TE_combine+'|\w+|[.,!?:;“”]'
    itr_lst = re.findall(find_regex, user_paragraph[i][0])
    all_lst.append(itr_lst)

def sep_punctuation(user_paragraph):
    itr_sep_punctuation = []
    lst_sep_punctuation = []
    for i in range(len(user_paragraph)):
        find_regex = r'\w+|[.,!?:;“”]'
        itr_sep_punctuation = re.findall(find_regex, user_paragraph[i][0])
        lst_sep_punctuation.append(itr_sep_punctuation)
    return lst_sep_punctuation

In [4]:
import re
   
def foobar_sec(string, item):
    first_string = []
    second_string = []
    blank_string = []
    if item in string:
        first_index = string.index(item) + len(item) + 1
        first_string = string[:first_index-1]
        second_string.append(string[first_index-1:])
    if second_string == ['']:
        return blank_string
    return second_string

def foobar_rev(string, item):
    first_string = []
    second_string = []
    revised_string = []
    if item in string:
        first_index = string.index(item) + len(item) + 1
        first_string = string[:first_index-1]
        second_string = string[first_index-1:]
        reg_item = r''+item+'|\w+|[.,!?:;“”]'
        revised_string = re.findall(reg_item, first_string)
    return revised_string

def remove_space(string):
    new_string = ""
    space = " "
    if space in string[0]:
        new_string = string[1:]
    else:
        new_string = string
    return new_string

def sep_pun(list):
    new_list = []
    new_list = re.findall(r"[\w']+|[.,!?:;“”]", list[0])
    return new_list

def flatten(t):
    return [item for sublist in t for item in sublist]

# token(string,item) takes two arguments
# string: list of list of string; the original sentence
# item: list of string; the token we want to apply to each word in sentence
def token(string,item):
    i = 0
    j = 0
    f0 = []
    f1 = []
    f2 = []
    f3 = []
    f4 = []
    f2_f3 = []
    new_count = []
    new_item = []
    count = 0
    final_list = []
    flag = j < len(string)
    while flag:
        while i < len(item):
            #print("\n---while loop start---\nj,i=", j,i)
            #print("string,item=",string[j],item[i],"\n")
            if item[i] in string[j][0]:
                f0 = foobar_rev(string[j][0],item[i])
                f2_f3.append(f0)
                f1 = foobar_sec(string[j][0],item[i])
                #print("---if item in string---\nitem=",item[i])
                #print("f0,f1",f0,f1,"\n")
                while i+1 < len(item):
                    if f1 != []:
                        f2 = foobar_rev(f1[0], item[i+1])
                        f3 = foobar_sec(f1[0], item[i+1])
                        f4 = f1
                        f4[0] = remove_space(f1[0])
                        f1 = f3
                        f2_f3.append(f2)
                        #print("---while i+1 < len---\nitem=",item[i+1],"\nf2,f3=",f2,f3)
                    if f2 == []:
                        j += 1
                        break  
                    i += 1
                i += 1
                f4 = sep_pun(f4)
                f2_f3.remove(f2)
                f2_f3.append(f4)
                #print("---f2_f3---\nf2_f3=",f2_f3)  
                #print("j,i---",j,i)
                if i+1 == len(item):
                    f1[0] = remove_space(f1[0])
                    f1 = sep_pun(f1)
                    f2_f3.append(f1)
                    print(f2_f3)
                    break
            else:
                i += 1
        if i == len(item):
            flag = False
    f2_f3 = flatten(f2_f3)
    return f2_f3

In [5]:
# complete token for each verse in the paragraph with numbers and punctuations
user_t = token(user_paragraph,TE)

In [6]:
# get number index for user_t
# print complete color-coded user paragraph with line break
user_t_index = get_index(user_t,num_lst)
for j in range(len(parsing)):
    for i in range(len(user_t)):
        if user_t[i] in parsing[j]:
            user_t[i] = code_to_color(parsing[j][-1])+parsing[j][0]+RESET 
            break
            
final_user = word_to_sentence(user_t,user_t_index)
for i in range(len(final_user)):
    print(final_user[i][0])

1 Now in those days a decree went out from Caesar Augustus , that a census be taken of all the inhabited earth .
2 This was the first census taken while Quirinius was governor of Syria .
3 And everyone was on his way to register for the census , each to his own city .
4 Joseph also went up from Galilee , from the city of Nazareth , to Judea , to the city of David which is called Bethlehem , because he was of the house and family of David ,
5 in order to register along with Mary , who was engaged to him , and was with child .
6 While they were there , the days were completed for her to give birth .
7 And she gave birth to her firstborn son ; and she wrapped Him in cloths , and laid Him in a manger , because there was no room for them in the inn .
8 In the same region there were some shepherds staying out in the fields and keeping watch over their flock by night .
9 And an angel of the Lord suddenly stood before them , and the glory of the Lord shone around them ; and they were terribly 